# Multiple rock types tracking

 
<div class="alert alert-block alert-danger">The option below has not really been tested and the performance of the code when using it will be really slow...</div>



***

<img src="images/rocktrack.png" alt="geometry" width="900" height="500"/>

_Above image shows the proportion of a specific rock (rock 1 ranging between 0 and 1) after 100,000 years. The initial model is composed of stratigraphic layers. A top one made only of 250 m of rock 0 and a bottom one made of rock 0 on half of the domain and of rock 1 on the other half. As the model starts, the top layer is first eroded and as the erosion progesses the second layer starts to be exposed and rock 1 starts to be deposited in the marine environment._


<img src="images/rock1.png" alt="geometry" width="500" height="500"/>

_Proportion of rock 1 in the sedimentary record which is higher than 40% (using Paraview)._

***

The approach consists in assigning multiple stratigraphic layers initially in the model and for each **node a specific rock type**. Multiple rock types can be defined on a single layer and there are no restriction on the number of types though the run time will become slower and slower has you increase it!

> One restriction is that on a single layer you will only be able to define for each point a single rock type. 

Basically you cannot have 50% of rock 1 and 50% of rock 2 on the one point of a single layer. But you can create 2 layers on top of each others one with only rock 1 and the second with rock 2. 

**Each layer has to be defined for all the nodes part of the computational domain but can have 0 thickness.**

To use the option you will need to add the following `XmL` element in the input file. 

```xml

    <!-- Erodibility structure
         This option allows you to specify different erodibility values either on the surface
         or within a number of initial stratigraphic layers. -->
    <erocoeffs>
        <!-- Active layer thickness [m]-->
        <actlay>5.</actlay>
        <!-- Stratal layer interval [a] -->
        <laytime>5000.</laytime>
        <!-- Number of rock types to track. -->
        <rocktype>2</rocktype>
        <!-- Definition of erodibility values for each rock types -->
        <rockero>
            <!-- Erodibility coefficient for rock type 1 -->
            <erorock>1.e-5</erorock>
        </rockero>
        <rockero>
            <!-- Erodibility coefficient for rock type 2 -->
            <erorock>3.e-5</erorock>
        </rockero>

        <!-- Number of erosion layers. -->
        <erolayers>2</erolayers>
        <!-- The layering is defined from top to bottom, with a file containing the
             following properties for each points of the regular grid (DEM):
              - 1st column the thickness of the considered layer
              - 2nd column the rock type ID (an integer) -->
        <!-- Constant erodibility and layer thickness -->
        <erolay>
            <!-- Uniform erodibility value for the considered layer. -->
            <laymap>data/thlay0.csv</laymap>
        </erolay>
        <erolay>
            <!-- Uniform erodibility value for the considered layer. -->
            <laymap>data/thlay1.csv</laymap>
        </erolay>
    </erocoeffs>
```

As seen above, we set 2 rocktypes, each rock is defined with a specific erodibility coefficient. Once eroded from their respective stratigraphic layer, all sediments will have an erodibility coefficient defined in the `<spl_law>` element of the `XmL` input file.   


I guess, in your case, if the idea is to track a rock from a particular catchment you can set the erodibility coefficient to the same values everywhere...

An example on how to create these stratigraphic layers is provided in the second notebook part of this folder:
+ `buildLayer.ipynb`

In [ ]:
import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action = "ignore", category = FutureWarning)

## Starting pyBadlands

First, we initialise the model and set the path to the **XmL** input file:

+ **input.xml**

> You can edit the **XmL** configuration file directly in the _Jupyter environment_ by clicking on it in the `tree` and changing in the **url** the **view** to **edit**.

In [ ]:
# Initialise model
model = badlandsModel()

# Define the XmL input file
model.load_xml('input.xml')

## Running pyBadlands

We can run the model for a given period. The end time in the **XmL** input file is set to _100 ky_ but you might want to run the model for a coupled of iterations and check the output before running the model for the entire simulation time. This is done by putting the time in the **run_to_time** function. 

In [ ]:
model.run_to_time(100000)

## Stratigraphic mesh

Once the model has finished you will need to build the stratigraphic mesh. The following cells are used for this purpose... again depending of the size of your model this can be really slow...

In [ ]:
from scripts import stratiMesh as strati

The stratigraphic mesh will be store in the same folder as your output file (defined in the `XmL` input file) and will be call using the given `xdmfName` (in the example below: `stratal_series`). The newly created xdmf file can be open in `Paraview` like the tin.series and flow.series files... 

First we initialise the function:

In [ ]:
mesh = strati.stratiMesh(folder='multiplerocks', xdmfName = 'stratal_series', ncpus=1, layperstep=1, dispTime=5000.)

Then we create the stratigraphic mesh for each stratigraphic layers:

In [ ]:
mesh.outputSteps(startTime=0.,endTime=100000.)